In [3]:
!pip install spatialmath-python > /dev/null

In [44]:
%matplotlib notebook
from spatialmath import *
from spatialmath.base.transforms3d import *
import matplotlib.pyplot as plt
from sympy import pi
import numpy as np
import sympy as sp

In [5]:
# extendemos la clase SE3 para poder manejar variables en forma simbolica y numérica
# USO: objeto.symbolReplace([(simbolo1, valor_simbolo1),(simboloN, valor_simboloN)])
class SE3(SE3):
    def symbolReplace(self,symbol_values):
        aux = np.eye(self.A.shape[0])
        for i in range(self.A.shape[0]):
            for j in range(self.A.shape[1]):
                try:
                    aux[i,j] = self.A[i,j].subs(symbol_values).evalf()
                except:
                    pass
        return aux

***Problema***

Tomemos el robot de 2GDL y colocamos ternas en sus extremos. Sea S0 su origen, S1 el extremo del primer eslabon y S2 el extremo del segundo. Suponiendo que sus variables articulares [Q] = [90,90] y sus dimensiones [L] = [2,10], obtener el modelo cinemático.

 °------L1------°-----L2------°

S0 - S1 - S2

In [18]:
# Definimos simbolos a utilizar
q1,l1, q2, l2 = sp.symbols('Q1,L1,Q2,L2')
S0 = SE3()

In [19]:
S1 = SE3.Rz(q1) * SE3.Tx(l1)
S1

  cos(Q1)      -sin(Q1)     0            L1*cos(Q1)    
  sin(Q1)      cos(Q1)      0            L1*sin(Q1)    
   0         0         1        0             
   0         0         0        1             


In [20]:
S1_eval = S1.symbolReplace([(q1,pi/2),(l1,2)])
S1_eval.round()

array([[ 0., -1.,  0.,  0.],
       [ 1.,  0.,  0.,  2.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [21]:
S2 = SE3.Rz(q2) * SE3.Tx(l2)
S2

  cos(Q2)      -sin(Q2)     0            L2*cos(Q2)    
  sin(Q2)      cos(Q2)      0            L2*sin(Q2)    
   0         0         1        0             
   0         0         0        1             


In [23]:
T = S1 * S2
T

  -sin(Q1)*sin(Q2) + cos(Q1)*cos(Q2) -sin(Q1)*cos(Q2) - sin(Q2)*cos(Q1) 0            L1*cos(Q1) - L2*sin(Q1)*sin(Q2) + L2*cos(Q1)*cos(Q2)  
  sin(Q1)*cos(Q2) + sin(Q2)*cos(Q1) -sin(Q1)*sin(Q2) + cos(Q1)*cos(Q2) 0            L1*sin(Q1) + L2*sin(Q1)*cos(Q2) + L2*sin(Q2)*cos(Q1)  
  0            0            1            0             
  0            0            0            1             


In [24]:
T = T.simplify()
T

  cos(Q1 + Q2) -sin(Q1 + Q2) 0            L1*cos(Q1) + L2*cos(Q1 + Q2)  
  sin(Q1 + Q2) cos(Q1 + Q2) 0            L1*sin(Q1) + L2*sin(Q1 + Q2)  
  0            0            1            0             
  0            0            0            1             


In [25]:
T_eval = T.symbolReplace([(q1,pi/2),(l1,2),(q2,pi/2),(l2,10)]).round()
T_eval

array([[ -1.,   0.,   0., -10.],
       [  0.,  -1.,   0.,   2.],
       [  0.,   0.,   1.,   0.],
       [  0.,   0.,   0.,   1.]])

In [32]:
p = T.t
# p=T.A[:3,3]
p

array([L1*cos(Q1) + L2*cos(Q1 + Q2), L1*sin(Q1) + L2*sin(Q1 + Q2), 0],
      dtype=object)

In [33]:
px,py,pz=p

In [34]:
px

L1*cos(Q1) + L2*cos(Q1 + Q2)

In [35]:
px.subs([('Q1',pi/2)])

-L2*sin(Q2)

In [36]:
py

L1*sin(Q1) + L2*sin(Q1 + Q2)